In [40]:
import pandas as pd
import numpy as np
import re

In [41]:
def join_columns(df, columns, prefix='_'):
    if prefix == '_':
        prefix = ''
    else:
        prefix += '_'
    value_set = set()
    df_row_values = []
    
    for row in range(len(df)):
        row_values = set()
        for col in columns:
            try:
                val = re.sub('\s+', '_', df.iloc[row][col].strip())
                
                for value in val.split(','):
                    value_set.add(value.lower().strip('_'))
                    row_values.add(value.lower().strip('_'))
            except AttributeError as e:
                pass
        df_row_values.append(row_values)
    
    df_copy = df.copy()
    df_copy[[prefix+x for x in value_set]] = 0
    
    for i, row in enumerate(df_row_values):
        for col in row:
            df_copy.loc[i, prefix+col] = 1
    
    print(columns)
    print(value_set)            
    df_copy = df_copy.drop(columns=columns)
    
    return df_copy
                
                

In [42]:
drop_columns = ['id', 'slug', 'thc', 'cbd']

products = pd.read_csv("./Data/kushy_datasets/Products/products-kushy_api.2017-11-14.csv")
products['strain'] = products['strain'].str.replace('-', ' ')
products = products.drop(columns=drop_columns)

strains = pd.read_csv("./Data/kushy_datasets/Strains/strains-kushy_api.2017-11-14.csv")
strains = strains.rename(columns={"name": "strain"})
strains['strain'] = strains['strain'].str.replace('-', ' ')
strains = strains.drop(columns=drop_columns)

df = products.merge(strains, how='left', on='strain', suffixes=('_prod_db', '_strain_db'))

ratings = pd.read_csv("./med_cabinet/Data/cannabis.csv", na_values=[0, '0'])
ratings['Strain'] = ratings['Strain'].str.replace('-', ' ')
ratings['Strain'] = ratings['Strain'].str.replace('98 white widow', 'white widow')
ratings = ratings.rename(columns={"Strain": "strain"})

df = df.merge(ratings, how='left', on='strain', suffixes=('_', '_ratings_db'))

df = join_columns(df, ['Flavor', 'flavor'], 'flavor')
df['flavor_tree_fruit'] = df['flavor_tree'] & df['flavor_fruit']
df = df.drop(columns=['flavor_tree', 'flavor_fruit'])
df['flavor_blue_cheese'] = df['flavor_blue'] & df['flavor_cheese']
df = df.drop(columns=['flavor_blue'])

df = join_columns(df, ['effects', 'Effects'], 'effect')
df = join_columns(df, ['ailment'], 'ailment')
df = join_columns(df, ['category'], 'category')
df = join_columns(df, ['Type', 'type'], 'type')

if 'none' in df.columns:
    df = df.drop(columns=['none'])
df = df.drop(columns=['crosses', 'image', 'description', 'status', 'sort', 
                 'lab_test', 'breeder', 'location', 'terpenes', 'thca', 'thcv', 
                 'cbda', 'cbdv', 'cbn', 'cbg', 'cbgm', 'cbgv', 'cbc', 
                 'cbcv', 'cbv', 'cbe', 'cbt', 'cbl', 'category_'])

df = df.rename(columns={
                'type_outdoo':'type_outdoor', 
                'type_organi':'type_organic', 
                'type_inorga':'type_inorganic', 
                'Rating':'rating', 
                'Description':'description', 
                'flavor_spicy/herbal':'flavor_spicy_herbal'})


['Flavor', 'flavor']
{'pungent', 'chestnut', 'earthy', 'tobacco', 'flowery', 'none', 'chemical', 'lime', 'ammonia', 'mint', 'coffee', 'apricot', 'mango', 'pear', 'citrus', 'sweet', 'pineapple', 'peach', 'lemon', 'tar', 'woody', 'diesel', 'vanilla', 'plum', 'butter', 'apple', 'strawberry', 'pepper', 'berry', 'blueberry', 'grapefruit', 'blue', 'fruit', 'minty', 'skunk', 'sage', 'tree', 'cheese', 'lavender', 'violet', 'nutty', 'tropical', 'grape', 'rose', 'orange', 'tea', 'pine', 'menthol', 'honey', 'spicy/herbal'}
['effects', 'Effects']
{'sleepy', 'none', 'talkative', 'euphoric', 'creative', 'energetic', 'dry', 'tingly', 'horny', 'hungry', 'uplifted', 'dry_mouth', 'paranoid', 'aroused', 'happy', 'giggly', 'focused', 'mouth', 'anxious', 'relaxed'}
['ailment']
{'muscle_spasms', 'insomnia', 'stress', 'inflammation', 'pain', 'nausea', 'depression', 'lack_of_appetite'}
['category']
{'', 'topical', 'concentrate', 'salt', 'kief', 'shatter', 'vape_cartidge', 'oil', 'drink', 'soup', 'edibles', 'v

In [43]:
df

,name,brand,strain,rating,description,flavor_pungent,flavor_chestnut,flavor_earthy,flavor_tobacco,flavor_flowery,...,category_chocolate,category_snack,category_wax,category_bath,category_spread,category_bubble_hash,category_pill,type_indica,type_hybrid,type_sativa
0,Pre-Roll Package - Pre-roll,The Humboldt Cure,NaN,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,ounce of cannabis,The Humboldt Cure,NaN,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1/2 ounce of cannabis,The Humboldt Cure,NaN,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1/4 ounce of cannabis,The Humboldt Cure,NaN,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1/8,The Humboldt Cure,NaN,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17377,Blueberry - Oil,PopNaturals,Blueberry,4.3,A true A-List cannabis strain. Blueberry’s leg...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
17378,Purple Haze - Oil,PopNaturals,Purple Haze,4.3,"Popularized by Jimi Hendrix’s 1967 classic, Pu...",0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,1
17379,Cannatonic - Oil,PopNaturals,Cannatonic,4.3,Cannatonic is a unique hybrid strain bred by S...,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
17380,Pineapple Kush - Wax,Unknown Producer,Pineapple Kush,4.2,Pineapple Kush marijuana strain has an aroma o...,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [45]:
df.to_csv('./Data/weed.csv', index=False)